In [1]:
#20240507

In [ ]:
'''
<선형회귀를 구현>
토이 데이터셋
▪ 실험용으로 만든 작은 데이터셋

예측 모델
▪ x와 y라는 두 개의 변수로 구성된 데이터 셋을 생성하며 둘은 선형관계
▪ x값이 주어지면 y값을 예측하는 모델을 만드는 것이며 y에 추가된 노이즈 때문에 점들이 구름같은 양상을 띈다.

코드:
'''
import numpy as np

np.random.seed(0)
x = np.random.rand(100,1)
y = 5+2*x+np.random.rand(100,1)

In [ ]:
'''
예측 모델의 목표
▪ 주어진 데이터를 잘 표현하는 함수 찾기
▪ y와 x가 선형 관계라고 가정하여, y = Wx + b 식으로 표현할 수 있음
▪ 데이터와 예측치의 차이는 잔차(residual)인데, 이를 최소화해야한다. 

평균 제곱 오차 (Mean Squared Error)
▪ 예측치(모델)와 데이터의 오차를 나타내는 지표
▪ 표현되는 손실 함수의 출력을 최소화하는 W와 b를 찾는 함수의 최적화 문제이다.
▪ 경사하강법을 사용하여 최소화하는 매개변수를 찾는다.

데이터의 예측치를 구하는 predict 함수 구현
▪ 매개변수 W와 b를 Variable 인스턴스로 생성
▪ matmul 함수를 사용하여 행렬의 곱을 계산
▪ X와 W의 대응하는 차원의 원소 수가 일치(D차원인 경우 W의 형상이 (D, 1)), y 형상은 (100, 1) 임
코드:
'''
W = Variable(np.zeros(1,1))
b = Variable(np.zeros(1))

def predict(x):
y = F.matmul(x, W) +b
return y

'''
▪ x.shape[1] 과 W.shape[0]을
일치시켜야 행렬 곱이 제대로 계산됨
▪ 100개의 데이터 각각에 대해 W에 의한
벡터의 내적을 계산
'''

In [ ]:
'''
평균 제곱 오차를 구하는 mean_squared_error 함수 구현
코드:
def mean_sqaured_error(x0, x1):
diff = x0 -x1
return F.sum(diff ** 2) / len(diff)

경사하강법으로 매개변수 갱신
▪ 매개변수를 갱신할 때 W.data -= lr * W.grad.data 처럼 인스턴스 변수의 data에 대한 계산
▪ 매개변수 갱신은 단순히 데이터를 갱신함
▪ 코드를 실행하면, 손실함수의 출력값이
줄어드는 것을 확인할 수 있음
▪ W = Variable([2.11807369]), b = Variable([5.46608905]) 라는 값을 얻음


<최종코드>
'''
import numpy as np
import matplotlib.pyplot as plt
from dezero import Variable
import dezero.functions as F

#토이데이터 설정
np.random.seed(0)
x = np.random.rand(100,1)
y = 5+2*x+np.random.rand(100,1)
x, y =Variable(x), Variable(y)

W = Variable(np.zeros((1,1)))
b = Variable(np.zeros(1))

def predict(x):
y = F.matmul(x, W) +b
return y

def mean_sqaured_error(x0, x1):
diff = x0 -x1
return F.sum(diff ** 2) / len(diff)

lr = 0.1
iters = 100

for i in range(iters):
y_pred = predict(x)
loss = mean_squared_errpor(y, y_pred)

W.cleargrad()
b.cleargrad()
loss.backward()

W.data -= lr * W.grad.data
b.data -= lr * b.grad.data
print(W, b, loss)


In [ ]:
'''
mean_squared_error 함수 구현의 문제점
▪ 중간에 등장하는 이름 없는 변수 세 개는 계산 그래프가 존재하는 동안 메모리에 계속 살아있음. 
▪ 이 변수들의 데이터 (ndarray 인스턴스)도 마찬가지로 계속 살아 있음
▪ 컴퓨터의 메모리가 충분하다면 지금의 구현 방식도 문제 없음
▪ 메모리 최적화를 위해 더 나은 방식을 도입해야함

-> 함수 개선.

mean_squared_error 함수 개선
▪ Function 클래스를 상속하여 구현하는 방식
▪ 순전파 , ndarray 인스턴스, 역전파를 구현
• mean_squared_error 함수에서 구현한 코드와 거의 동일
• 수식으로 미분을 계산한 다음 해당 수식을 코드로 구현
▪ 중간에 등장하던 변수들이 사라짐
(forward 메서드에서만 사용됨)
▪ 새로 구현한 mean_squared_error 함수는 앞서 구현한 함수와 같은 결과를 얻지만 메모리는 덜 사용함

코드:
'''
class MeanSquaredError(Function):
def forward(self, x0, x1):
diff = x0 - x1
y = (diff ** 2).sum() / len(diff)
return y

def backward(self, gy):
x0, x1 = self.inputs
diff = x0 - x1
gx0 = gy * diff * (2. / len(diff))
gx1 = -gx0
return gx0, gx1

def mean_sqaured_error(x0, x1):
return MeanSquaredError()(x0,x1)

In [ ]:
'''
<선형 회귀 구현을 신경망으로 확장>

선형 변환 혹은 아핀 변환(Affine Transformation) 
▪ 입력 x와 매개변수 W 사이에서 행렬 곱을 구하고 거기에 b를 더함
▪ 선형 변환은 신경망에서는 완전연결계층에 해당.
선형 변환을 linear 함수로 구현하는 방식
▪ Function 클래스를 상속하여 새롭게 Linear라는 함수 클래스를 구현.
▪ Linear 함수 클래스 구현 방식이 메모리를 더 효율적으로 사용함.
▪ DeZero의 matmul 함수와 +(add 함수)를 이용한 방식: matmul 함수의 출력은 Variable 인스턴스이므로 계산 그래프에 기록됨
▪ Function 클래스를 상속하여 Linear 클래스를 구현하는 방식: 중간 결과가 Variable 인스턴스로 보존되지 않기 때문에 순전파 시 사용하던 중간 데이터는 순전파가 끝나는 즉시 삭제됨.

DeZero 의 함수를 사용하면서 메모리 효율 개선하는 방법
▪ 변수 t는 matmul 함수의 출력인 동시에 +(add) 함수의 입력. +의 역전파는 출력 쪽의 기울기를 단순히 흘려보낸다.
▪ matmul 역전파는 입력 x, W, b 만 사용, t의 데이터는 + 역전파와 matmul 역전파에 필요로 하지 않음
▪ 기울기를 흘려보내야 하므로 계산 그래프에서는 변수 t가 필요하지만, 그 안의 데이터는 즉시 지워도 상관X.


linear_simple 함수 구현
▪ 인수 x와 W는 Variable 인스턴스 혹은 ndarray 인스턴스라고 가정
▪ ndarray 인스턴스라면 matmul 함수 안에서 Variable 인스턴스로 변환
▪ B = None 인 경우는 단순히 행렬 곱셈만 계산하여 결과를 반환
▪ 편향이 주어지면 더해준다.
▪ t.data = None 코드에서 t 데이터를 메모리에서 삭제.

코드:
'''
def linear_simple(x,W,b=None):
t = matmul(x,W)
if b is None:
return t

y= t + b
t.data = None #t데이터를 메모리에서 삭제
return y


In [ ]:
'''
 비선형 데이터셋
▪ 선형 회귀로는 문제를 풀수 없는 데이터셋이지만 신경망을 이용하여 해결할 수 있다.

 비선형 데이터 셋 생성
▪ sin 함수를 사용하여 데이터를 생성
▪ 생성한 (x, y) 점들을 2차원 평면에 그려보면 ^v의 형태.

코드: 
'''
import numpy as np

np.random.seed(0)
x = np.random.rand(100, 1)
y = np.sin(2 * np.pi * x) + np.random.rand(100,1)

In [ ]:
'''
활성화 함수
▪ 선형 변환은 입력 데이터를 선형으로 변환해 줌
▪ 신경망은 선형 변환의 출력에 비선형 변환을 수행한다. 그리고 비선형 변환이 활성화 함수이다. ( ReLU, 시그모이드 함수 등이 있음)

시그모이드 함수
▪ 활성화 함수로 시그모이드 함수를 사용하여 신경망을 구현하고 이 비선형 변환이 텐서의 각 원소에 적용된다.

시그모이드 함수 코드:
'''
def sigmoid_simple(x):
x = as_variable(x)
y = 1 / (1 + exp(-x))
return

In [ ]:
'''
신경망 구현 : 신경망 추론 코드
▪ 일반적인 신경망 형태는 연속적으로 변환을 수행한다.
(선형 변환 -> 활성화 함수 -> 선형 변환 -> 활성화 함수 ->….)
▪ 선형 변환과 활성화 함수를 순서대로 적용
▪ 추론의 정확성을 높이려면 학습이 필요함
▪ 신경망 학습에서는 추론을 처리한 후 손실 함수를 추가한다.
▪ 손실 함수의 출력을 최소화하는 매개변수를 찾아야하며 선형 변환이나 활성화 함수 등에 의한 변환을 층(Layer)라고 한다.


실제 데이터셋을 활용하여 신경망 학습
(1) 매개변수 초기화
• I는 입력층의 차원 수, H는 은닉층의 차원 수, O는 출력층의 차원 수, 편향은 0 벡터로 초기화
(2) 신경망 추론을 수행
(3) 매개변수 갱신
▪ 비선형 관계도 제대로 학습함
▪ 이 구현 방식을 적용하여 더 깊은 신경망도 구현할 수 있다.

코드:
'''
import numpy as np
from dezero import Variable
import dezero.functions as F

np.random.seed(0)
x = np.random.rand(100,1)
y = np.sin(2 * np.pi * x)+np.random.rand(100,1)

#가중치 초기화
I, H, O = 1, 10, 1
W1 = Variable(0.01 * np.random.randn(I,H))
b1 = Variable(np.zeros(H))
W2 = Variable(0.01 * np.random.randn(H,O))
b2 = Variable(np.zeros(O))

#신경망 추론

def predict(x):
y = F.linear(x, W1, b1)
y = F.sigmoid(y)
y = F.linear(y, W2, b2)
return y

lr = 0.2
iters = 10000

#신경망 학습
for i in range(iters):
y_pred = predict(x)
loss = F.mean_squared_error(y, y_pred)

W1.cleargrad()
b1.cleargrad()
W2.cleargrad()
b2.cleargrad()
loss.backward()

W1.data -= lr * W1.grad.data
b1.data -= lr * b1.grad.data
W2.data -= lr * W2.grad.data
b2.data -= lr * b2.grad.data
if i % 1000 == 0:
print(loss)


In [ ]:
'''
DeZero 신경망 프레임워크의 사용 편의성 개선
▪ 지금의 DeZero는 사용 편의성 면에서 몇 가지 문제가 있음
▪ 층이 깊어 질수록 매개변수 관리가 번거로워 짐
( 매개변수의 기울기를 재설정하거나 매개변수를 갱신하는 등의 작업)
▪ 매개변수 관리를 간소화하는 구조 필요
( 더욱 복잡한 신경망 구현시 매개변수를 다루는 일도 그 만큼 복잡해 짐) 
->매개변수를 담는 구조를 만듬 : Parameter와 Layer 라는 클래스를 구현하여 매개변수 관리를 자동화할 수 있음

Parameter 클래스
▪ Parameter 클래스는 Variable 클래스와 똑같은 기능을 갖게 함
▪ Variable 클래스를 상속한 것 뿐이라 기능도 Variable 클래스와 동일함

코드:
class Parameter(Variable):
pass

Parameter 인스턴스와 Variable 인스턴스 구별
▪ Parameter 인스턴스와 Variable 인스턴스를 조합하여 계산할 수 있으며 isinstance 함수로 구분할 수 있음
▪ 이 점을 이용하여 Parameter 인스턴스만을 담는 구조를 만들 수 있다.

코드:
import numpy as np
from dezero import Variable, Parameter

x = Variable(no.array(1.0))
y = Parameter(np.array(2.0))
y = x * p

print(isinstance(p, Parameter))
print(isinstance(x, Parameter))
print(isinstance(y, Parameter))

결과:
TRUE
FALSE
FALSE

'''

In [ ]:
'''
Layer 클래스
▪ Function 클래스와 마찬가지로 변수를 변환하는 클래스
▪ 매개변수를 유지한다는 점이 다르다. Layer 클래스를 기반 클래스로 두고 구체적인 변환은 자식 클래스에서 구현함
Layer 클래스 구현
1)
▪ _params 라는 인스턴스 변수는 Layer 인스턴스에 속한 매개변수를 보관.
▪ __setattr__ 메서드는 인스턴스 변수를 설정할 때 호출.
▪ 이 메서드를 override 하면 인스턴스 변수를 설정할 때 커스텀 로직을 추가할 수 있음.

코드:
from dezero.core import Parameter

class Layer:
def __init__(self):
self._params = set()

def __setattr__(self, name, value):
if isinsrance(value, Parameter):
self._params.add(name)
super().__setattr__(name, value)

2) 매개변수를 인스턴스 변수 _params 저장
▪ value가 Parameter 인스턴스라면 self._params에 name을 추가함
▪ name을 추가해 Layer 클래스가 갖는 매개변수를 인스턴수 변수 _params에 모아둘 수 있음

layer = Layer()

layer.p1 = Parameter(np.array(1))
layer.p2 = Parameter(np.array(2))
layer.p3 = Parameter(np.array(3))
layer.p4 = 'test'

print(layer._params)
print('--------------')

for name in layer._params:
print(name, layer.__dict__[name])


결과:
{'p2', 'p1'}
--------
p2 variable(2)
p1 variable(1)



▪ layer 인스턴스 변수를 설정하면 Parameter 인스턴스를 보유하고 있는 변수 이름만 layer._params에 추가됨
▪ 인스턴스 변수 __dict__에는 모든 인스턴스 변수가 딕셔너리 타입으로 저장되므로 Parameter 인스턴스만 꺼낼 수 있음

(3)Layer 클래스의 4개의 메소드 추가
__call__ 메서드
• 입력받은 인수를 건네 forward 메서드 호출
• 출력이 하나뿐이라면 튜플이 아니라 그 출력을 직접 반환
• 입력과 출력 변수를 약한 참조로 유지

params 메서드
• Layer 인스턴스에 담겨 있는 Parameter인스턴스들을 꺼내줌

cleargrads 메서드
• 모든 매개변수의 기울기를 재설정함

yield 반환
• 처리를 일시 중지하고 값을 반환
• yield 를 사용하면 작업을 재개할 수 있음

코드(class Layer 안에 구현):
'''
def __call__(self, *inputs):
outputs = self.forward(*inputs)
if not isinstance(outputs, tuple):
outputs = (outputs,)
self.inputs = [weakref.ref(x) for x in inputs]
self.outputs = [weakref.ref(y) for y in outputs]
return outputs if len(outputs) > 1 else outputs[0]

def forward(self, inputs):
raise NotImplementedError()

def params(self):
for name in self._params:
yield self.__dict__[name]

def cleargrads(self):
for params in self.params():
params.cleargrad()

In [ ]:
'''
선형 변환을 하는 Linear 클래스 구현
▪ 계층으로서의 Linear 클래스를 구현
▪ Linear 클래스는 Layer 클래스를 상속하여 구현
▪ 가중치와 편향은 두 Parameter 인스턴스 변수의 이름이 self._params에 추가됨
▪ forward 메서드로 선형 변환을 구현 (linear 함수를 호출)

코드:
'''
class Linear(Layer):
def __init__(self, in_size, out_size, nobias=False, dtype=np.float32):
super().__init__()

I, O = in_size, out_size
W_data = np.random.randn(I, O).astype(dtype) * np.sqrt(1 / I)
self.W = Parameter (W_data, name ='W')
if nobias:
self.b = None
else:
self.b = Parameter(np.zeros(0, dtype=dtype), name='b')

def forward(self,x):
 y = F.linear(x, self.W, self.b)
return y


In [ ]:
'''
Linear 클래스 구현하는 더 나은 방법
▪ 가중치 W를 생성하는 시점을 늦추는 방식: 가중치를 초기화 메서드가 아닌 forward 메서드에서 생성함으로써 Linear 클래스의 입력 크기를 자동으로 결정할 수 있음
▪ __init__ 메서드에서 in_size를 지정하지 않아도 됨
▪ forward(self, x) 메서드에서 입력 x의 크기에 맞게 가중치 데이터를 생성
▪ layer = Linear(100) 처럼 출력 크기만 지정해도 됨

Linear 클래스를 이용하여 신경망을 구현
▪ 이전 문제 sin 함수의 데이터셋에 대한 회귀 문제를 다시 풀어봄
▪ 주목할 점은 매개변수 관리를 Linear 인스턴스가 맡고 있음
▪ 매개변수 기울기 재설정과 매개변수 갱신 작업이 전보다 깔끔해짐
▪ Linear 클래스를 개별적으로 다루는 부분이 추후 깊은 신경망 구현을 위해 개선되어야 함.

(기존 코드는 건너뛰고 확장 버전의 코드를 다루겠음)

확장된 새로운 Layer 클래스 구조
▪ Layer 클래스는 매개변수를 관리하는 구조라 이를 사용하면 매개변수를 사용자가 직접 다루지 않아도 되어 편리하지만 마찬가지로 Layer 인스턴스 자체도 관리가 필요하다.(10층 신경망을 구현하려면 10개의 Layer 인스턴스를 관리해야 함)
▪ 확장된 Layer 클래스는 여러 개의 Parameter를 가질 수 있다.
▪ Layer 클래스 안에 다른 Layer 가 들어가는 구조이며 바깥 Layer 에서 그 안에 존재하는 모든 매개변수를 꺼낼 수 있도록 한다.
▪ 첫 번째 변화: 인스턴스 변수를 설정할 때 Layer 인스턴스의 이름도 _params에 추가하도
록 함
▪ 두 번째 변경점: 매개변수를 꺼내는 처리를 name에 해당하는 객체 obj 가 Layer 인스턴스라면 obj.params() 을 호출, Layer 속 Layer에서도 매개변수를 재귀적으로 꺼낼 수 있도록 한다.

새로운 Layer 클래스를 사용하여 신경망 구현
▪ model = Layer( )에서 인스턴스 생성한 다음 model 인스턴스 변수로 Linear 인스턴스 추가
▪ model.params( )로 model 내에 존재하는 모든 매개변수에 접근할 수 있음
▪ model.cleargrads( )는 모든 매개변수의 기울기를 재설정함

코드:
'''
class Layer:
def __init__(self):
self._params = set()

def __setattr__(self, name, value):
if isinstance(value, (Parameter, Layer)):
self._params.add(name)
super().__setattr__(name, value)

def params(self):
for name in self._params:
obj = self.__dict__[name]

if isinstance(obj, Layer):
yield from obj.params()
else:
yield obj


###########################################################

import dezero.layers as L
import dezero.functions as F
from dezero import Layer

model = Layer()
model = L.Linear(5)
model = L.Linear(3)

def predict(model, x):
y = model.l1(x)
y = F.sigmoid(y)
y = model.12(y)
return y

#모든 매개변수에 접근
for p in model.params():
print(p)

#모든 매개변수의 기울기를 재설정
model.cleargrads()

In [ ]:
'''
Layer 클래스를 더 편리하게 사용하는 방법
▪ Layer 클래스를 상속하여 모델 전체를 하나의 클래스로 정의하는 방법
▪ TwoLayerNet 이름으로 클래스 모델을 정의하낟. 이 클래스는 Layer를 상속함
▪ __init__ 메서드에서는 필요한 Layer들을 생성하여 self.l1.. 형태로 설정
▪ forward 메서드에는 추론을 수행하는 코드를 작성
▪ TwoLayerNet 클래스 하나에 신경망에 필요한 모든 코드를 집약할 수 있음

코드:
'''
class TwoLayerNet(Model):
def __init__(self, hidden_size, out_size):
super().__init__()
self.l1 = L.Linear(hidden_size)
self.l2 = L.Linear(out_size)

def forward(self,x):
y = F.sigmoid(self.l1(x))
y = self,l2(y)
return y

In [ ]:
'''
모델을 표현하기 위한 Model 클래스 생성
▪ 신경망도 수식으로 표현할 수 있는 함수이며 그것을 가르켜 모델이라고 함
▪ Model 클래스는 Layer 클래스의 상속
▪ 시각화를 위한 plot 메서드는 인수 *inputs로 전달받은 데이터를 forward 메서드로 계산 후
생성된 계산 그래프를 이미지 파일로 내보냄
▪ Model 클래스는 마치 Layer 클래스처럼 활용할 수 있음

코드:
'''
from dezero import Variable, Model
import dezero.layers as L
import dezero.functions as F

class TwoLayerNet(Model):
def __init__(self, hidden_size, out_size):
super().__init__()
self.l1 = L.Linear(hidden_size)
self.l2 = L.Linear(out_size)

def forward(self, x):
y = F.sigmoid(self.l1(x))
y = self.l2(y)
return y

x = Variable(np.random.randn(5, 10), name('x'))
model = TwoLayerNet(100, 10)
model.plot(x)

#model 클래스를 사용한 문제해결
model = TwoLayerNet(hidden_size, 1)

#학습 시작
for i in range(max_iter):
y_pred = model(x)
loss = F.mean_squared_error(y,y_pred)

model.cleargrad()
loss.backward()

for p in model.params():
p.data -= lr * p.grad.data
if i % 1000 == 0:
print(loss)

In [ ]:
'''
이외의클래스: MLP -완전연결계층 신경망 구현
▪ fc_output_sizes는 신경망을 구성하는 완전연결 계층들의 출력 크기를 튜플 또는 리스트로 지정함
((10, 1)을 건네면 2개의 Linear 계층을 만들고
첫 번째 계층의 출력 크기는 10, 두번째 계층 출력 크기는 1로 구성)
▪ 계층 모델에 인스턴스 변수를 설정하는 식으로 계층에 포함된 매개변수들을 관리함
'''